In [ ]:
def get_data(endpoint, params):
    response = requests.get(
        URL+endpoint,
        headers = headers,
        params = params
    )
    if response.status_code == 200:
            
        remaining = response.headers.get("x-ratelimit-requests-remaining")
        data = response.json()
        print(f"requests before reaching limit {remaining}")

    else:
        print(f"Error {response.status_code}, {response.text}")

    return data, remaining                                                                     

In [ ]:
def preprocess_players(data):
    player_info = data['player']
    stats = data['statistics'][0] if data['statistics'] else {}
    
    player_dict = {
        'player_id': player_info['id'],
        'player_name': player_info['name'],
        'firstname': player_info['firstname'],
        'lastname': player_info['lastname'],
        'age': player_info['age'],
        'nationality': player_info['nationality'],
        'height': player_info['height'],
        'weight': player_info['weight'],
        'injured': player_info['injured'],
        'team_id': stats.get('team', {}).get('id', None),
        'team_name': stats.get('team', {}).get('name', None),
        'league_id': stats.get('league', {}).get('id', None),
        'season': stats.get('league', {}).get('season', None),
        'appearences': stats.get('games', {}).get('appearences', None),
        'lineups': stats.get('games', {}).get('lineups', None),
        'minutes': stats.get('games', {}).get('minutes', None),
        'position': stats.get('games', {}).get('position', None),
        'rating': stats.get('games', {}).get('rating', None),
        'captain': stats.get('games', {}).get('captain', None),
        'substitutes_in': stats.get('substitutes', {}).get('in', None),
        'substitutes_out': stats.get('substitutes', {}).get('out', None),
        'substitutes_bench': stats.get('substitutes', {}).get('bench', None),
        'shots_total': stats.get('shots', {}).get('total', None),
        'shots_on': stats.get('shots', {}).get('on', None),
        'goals_total': stats.get('goals', {}).get('total', None),
        'goals_conceded': stats.get('goals', {}).get('conceded', None),
        'goals_assists': stats.get('goals', {}).get('assists', None),
        'goals_saves': stats.get('goals', {}).get('saves', None),
        'passes_total': stats.get('passes', {}).get('total', None),
        'passes_key': stats.get('passes', {}).get('key', None),
        'passes_accuracy': stats.get('passes', {}).get('accuracy', None),
        'tackles_total': stats.get('tackles', {}).get('total', None),
        'tackles_blocks': stats.get('tackles', {}).get('blocks', None),
        'tackles_interceptions': stats.get('tackles', {}).get('interceptions', None),
        'duels_total': stats.get('duels', {}).get('total', None),
        'duels_won': stats.get('duels', {}).get('won', None),
        'dribbles_attempts': stats.get('dribbles', {}).get('attempts', None),
        'dribbles_success': stats.get('dribbles', {}).get('success', None),
        'dribbles_past': stats.get('dribbles', {}).get('past', None),
        'fouls_drawn': stats.get('fouls', {}).get('drawn', None),
        'fouls_committed': stats.get('fouls', {}).get('committed', None),
        'cards_yellow': stats.get('cards', {}).get('yellow', None),
        'cards_yellowred': stats.get('cards', {}).get('yellowred', None),
        'cards_red': stats.get('cards', {}).get('red', None),
        'penalty_won': stats.get('penalty', {}).get('won', None),
        'penalty_committed': stats.get('penalty', {}).get('commited', None),
        'penalty_scored': stats.get('penalty', {}).get('scored', None),
        'penalty_missed': stats.get('penalty', {}).get('missed', None),
        'penalty_saved': stats.get('penalty', {}).get('saved', None)
    }
    
    return player_dict

In [ ]:
def preprocess_teams(data):
    team_info = data['team']
    
    team_dict = {
        'team_id': team_info['id'],
        'team_name': team_info['name'],
        'team_country': team_info['country'],
        'national': team_info['national'],
        'venue_capacity': data['vanue']['capacity'],
        'venure_surface': data['venue']['surface']
    }
    return team_dict

In [58]:
import time
import boto3
import pandas as pd

# get current and european seasons files from bucket
s3 = boto3.client('s3')

bucket_name = 'sport-storage'
object_key1 = 'current.csv'
object_key2 = 'european_seasons.csv'

response1 = s3.get_object(Bukcet = bucket_name, Key = object_key1)
response2 = s3.get_object(Bukcet = bucket_name, Key = object_key2)

current = pd.read_csv(io.BytesIO(resposne1['Body'].read()))
european_seasons = pd.read_csv(io.BytesIO(response2['Body'].read()))

league = current['league_id'][0]
year = current['year'][0]
to_find = current['type'][0]
page = current['page'][0]
index = np.where((european_seasons['league_id']==league)&(european_seasons['year']==year))[0][0]
remaining = 100
total_players_data = []
total_teams_data = []

while (index < len(european_seasons))&(remaining > 0):
    params = {'league_id': league,
              'season': year,
              'page': page
             }
    endpoint = to_find
    data, remaining_req = get_data(endpoint, params)
    
    #preprocess data
    if endpoint == 'players':
        players_data = [preprocess_players(data) for player in data['response']]
        total_players_data.append(players_data)
    else:
        teams_data = [preprocess_teams(data) for team in data['response']]
        total_teams_data.append(teams_data)
    
    max_page = df['paging']['total']
    
    if page == max_page & endpoint == 'players':
        to_find = 'teams'
        page = 1
        
    elif page == max_page & endpoint == 'teams':
        league = european_seasons.loc[index+1]['league_id']
        year = european_seasons.loc[index+1]['year']
        if european_seasons.loc[index+1]['players']:
            to_find = 'players'
        else:
            to_find = 'teams'
        page = 1
        
    else:
        page += 1
        
    remaining = remaining_req
    time.sleep(7)
#saving to sql database
if len(total_teams_data) > 0:
    
if len(total_players_data) > 0:


if index < len(european_seasons):
    data = {'league_id': [league], 'year': [year], 'type': [to_find], 'page': [page]}
    current = pd.DataFrame(data)
    current.to_csv('current.csv')
    print(f'{len(european_seasons)-index} seasons left')
else:
    print('All teams and players playing in european leagues collected')

Index([0], dtype='int64')


In [69]:
european_seasons.head()

,league_id,name,country,year,players,statistics_fixtures,current,predictions,odds
0,39,Premier League,England,2010,True,False,False,True,False
1,39,Premier League,England,2011,True,False,False,True,False
2,39,Premier League,England,2012,True,False,False,True,False
3,39,Premier League,England,2013,True,False,False,True,False
4,39,Premier League,England,2014,True,True,False,True,False
